In [204]:
import numpy as np

# configure formatting
np.set_printoptions(formatter={'float': lambda x: "{0:0.1f}".format(x)})


def sign(x):
    if x >= 0:
        return 1
    else:
        return -1

def householder(w):
    v = np.zeros_like(w)

    v[0] = w[0] + sign(w[0]) * np.linalg.norm(w)
    v[1:] = w[1:]
    
    H = np.eye(len(w)) - 2 * np.outer(v, v) / np.dot(v, v)
    
    return H


def qrDecomposition(A):
    (rows, columns) = A.shape
    
    Q = np.identity(rows)
    R = np.copy(A)

    for column in range(columns):
        # get needed vector and apply householder transformation
        y = R[column:, column]
        Hk = householder(y)

        # fill up into correct matrix dimension
        Qk = np.identity(rows)
        Qk[column:,column:] = Hk

        # apply to matrices
        Q = Qk@Q
        R = Qk@R

    # cut into needed shapes
    R = R[0:columns,:] 
    Q = Q.T[:,0:columns]

    return Q, R



In [205]:
# example testcase
A = np.array([  [-1,  7, -8, -9,  6],
                [-6, -8,  0,  3,  8],
                [-4, -2,  8,  0, -2],
                [-1, -9,  4, -8,  2],
                [-3, -5, -5,  7, -4],
                [-7, -4,  7, -1,  5],
                [-9, -7,  6, -5, -8],
                [-4, -3, -5,  3, -6],
                [ 5,  7,  5, -4, -5],
                [ 4, -6, -8, -2, -5]],dtype=float)

Q, R = qrDecomposition(A)
print(Q)
print(R)

assert(np.sum(A-Q@R) < 1e-8)

[[-0.1 0.5 0.5 -0.6 0.2]
 [-0.4 -0.2 0.1 0.1 0.5]
 [-0.3 0.1 -0.3 0.1 -0.1]
 [-0.1 -0.5 -0.2 -0.5 0.2]
 [-0.2 -0.2 0.3 0.4 -0.2]
 [-0.4 0.1 -0.2 -0.0 0.3]
 [-0.6 -0.0 -0.1 -0.3 -0.5]
 [-0.3 -0.0 0.4 0.1 -0.4]
 [0.3 0.2 -0.4 -0.1 -0.3]
 [0.3 -0.6 0.3 -0.2 -0.2]]
[[15.8 11.8 -6.5 -0.6 -1.3]
 [0.0 15.6 1.4 -1.8 3.1]
 [0.0 0.0 -18.0 2.9 0.9]
 [-0.0 -0.0 -0.0 15.7 -1.6]
 [0.0 0.0 0.0 0.0 16.9]]


In [206]:
Q1,R1 = np.linalg.qr(A)
print(Q1)
print(R1)

[[-0.1 0.5 0.5 -0.6 0.2]
 [-0.4 -0.2 0.1 0.1 0.5]
 [-0.3 0.1 -0.3 0.1 -0.1]
 [-0.1 -0.5 -0.2 -0.5 0.2]
 [-0.2 -0.2 0.3 0.4 -0.2]
 [-0.4 0.1 -0.2 -0.0 0.3]
 [-0.6 -0.0 -0.1 -0.3 -0.5]
 [-0.3 -0.0 0.4 0.1 -0.4]
 [0.3 0.2 -0.4 -0.1 -0.3]
 [0.3 -0.6 0.3 -0.2 -0.2]]
[[15.8 11.8 -6.5 -0.6 -1.3]
 [0.0 15.6 1.4 -1.8 3.1]
 [0.0 0.0 -18.0 2.9 0.9]
 [0.0 0.0 0.0 15.7 -1.6]
 [0.0 0.0 0.0 0.0 16.9]]
